<a href="https://colab.research.google.com/github/BahodirML/Coding_Practices/blob/main/YOLOv3_in_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python==4.2.0.34
!pip install opencv-python-headless==4.2.0.34


ERROR: Could not find a version that satisfies the requirement opencv-python==4.2.0.34 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80)
ERROR: No matching distribution found for opencv-python==4.2.0.34
ERROR: Could not find a version that satisfies the requirement opencv-python-headless==4.2.0.34 (from versions: 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80)
ERROR: No matching distribution found for opencv-python-headless==4.2.0.34


In [ ]:
# import the necessary packages
import numpy as np
import time
import cv2
import os
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt

# Define our imshow function
def imshow(title = "Image", image = None, size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

# # Download and unzip our images and YOLO files
# !wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/YOLO.zip
# !unzip -qq YOLO.zip

In [ ]:
import requests

# URLs for the YOLOv3 config and weights files
cfg_url = 'https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov3.cfg'
weights_url = 'https://pjreddie.com/media/files/yolov3.weights'
names_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names'

# Download yolov3.cfg
response = requests.get(cfg_url)
with open('yolov3.cfg', 'wb') as file:
    file.write(response.content)

# Download yolov3.weights
response = requests.get(weights_url)
with open('yolov3.weights', 'wb') as file:
    file.write(response.content)

# Download coco.names
response = requests.get(names_url)
with open('coco.names', 'wb') as file:
    file.write(response.content)

# Verify the downloads
import os
print(os.listdir('.'))


['.config', 'yolov3.cfg', 'yolov3.weights', 'coco.names', 'sample_data']


In [ ]:
# Load the COCO class labels our YOLO model was trained on
labelsPath = "coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

# We now need to initialize a list of colors to represent each possible class label
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

print("Loading YOLO weights...")

weights_path = "yolov3.weights"
cfg_path = "yolov3.cfg"

# Create our blob object
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)

# Set our backend
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
# net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

print("Our YOLO Layers")
ln = net.getLayerNames()

# There are 254 Layers
print(len(ln), ln)

Loading YOLO weights...
Our YOLO Layers
254 ('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36'

In [ ]:
import requests

# URLs for the YOLOv3 config and weights files
cfg_url = 'https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov3.cfg'
weights_url = 'https://pjreddie.com/media/files/yolov3.weights'
names_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names'

# Download yolov3.cfg
response = requests.get(cfg_url)
with open('yolov3.cfg', 'wb') as file:
    file.write(response.content)

# Download yolov3.weights
response = requests.get(weights_url)
with open('yolov3.weights', 'wb') as file:
    file.write(response.content)

# Download coco.names
response = requests.get(names_url)
with open('coco.names', 'wb') as file:
    file.write(response.content)

print("YOLOv3 configuration and weights files downloaded.")


YOLOv3 configuration and weights files downloaded.


In [ ]:
print("Starting Detections...")
# Get images located in ./images folder
mypath = "/content/images/"
file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Loop through images run them through our classifer
for file in file_names:
    # load our input image and grab its spatial dimensions
    image = cv2.imread(mypath+file)
    (H, W) = image.shape[:2]

    # we want only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # Now we contruct our blob from our input images
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    # We set our input to our image blob
    net.setInput(blob)
    # Then we run a forward pass through the network
    layerOutputs = net.forward(ln)

    # we initialize our lists for our detected bounding boxes, confidences, and classes
    boxes = []
    confidences = []
    IDs = []

    # Loop over each of the layer outputs
    for output in layerOutputs:

        # Loop over each detection
        for detection in output:
            # Obtain class ID and probality of detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # We keep only the most probably predictions
            if confidence > 0.75:
                # We scale the bounding box coordinates relative to the image
                # Note: YOLO actually returns the center (x, y) of the bounding
                # box followed by the width and height of the box
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # Get the the top and left corner of the bounding box
                # Remember we alredy have the width and height
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # Append our list of bounding box coordinates, confidences and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                IDs.append(classID)

    # Now we apply non-maxima suppression to reduce overlapping bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

    # We proceed once a detection has been found
    if len(idxs) > 0:
        # iterate over the indexes we are keeping
        for i in idxs.flatten():
            # Get the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # Draw our bounding boxes and put our class label on the image
            color = [int(c) for c in COLORS[IDs[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 3)
            text = "{}: {:.4f}".format(LABELS[IDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # show the output image
    imshow("YOLO Detections", image, size = 12)

Starting Detections...


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import requests

# Check if the necessary files are present
required_files = ['yolov3.cfg', 'yolov3.weights', 'coco.names']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"{file} not found. Please download the required YOLOv3 files.")

print("Starting Detections...")

# Path to images folder
mypath = "./images/"

# Get images located in ./images folder
file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Load YOLO model
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

# Load COCO class labels
with open('coco.names', 'r') as f:
    LABELS = f.read().strip().split('\n')

# Generate colors for each class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype='uint8')

# Loop through images and run them through our classifier
for file in file_names:
    # Load our input image
    image_path = join(mypath, file)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Failed to load image {image_path}")
        continue

    # Grab its spatial dimensions
    (H, W) = image.shape[:2]

    # Get the output layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # Construct our blob from our input images
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Run a forward pass through the network
    layerOutputs = net.forward(ln)

    # Initialize our lists for detected bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    IDs = []

    # Loop over each of the layer outputs
    for output in layerOutputs:
        # Loop over each detection
        for detection in output:
            # Obtain class ID and probability of detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Keep only the most probable predictions
            if confidence > 0.75:
                # Scale the bounding box coordinates relative to the image
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # Get the top and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # Append our list of bounding box coordinates, confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                IDs.append(classID)

    # Apply non-maxima suppression to reduce overlapping bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

    # Proceed once a detection has been found
    if len(idxs) > 0:
        # Iterate over the indexes we are keeping
        for i in idxs.flatten():
            # Get the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # Draw our bounding boxes and put our class label on the image
            color = [int(c) for c in COLORS[IDs[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 3)
            text = "{}: {:.4f}".format(LABELS[IDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show the output image
    cv2.imshow("YOLO Detections", image)
    cv2.waitKey(0)

cv2.destroyAllWindows()


Starting Detections...
Failed to load image ./images/image1.jpg


IndexError: invalid index to scalar variable.

In [ ]:
import os

# List all files in the 'images' directory
print("Files in 'images' directory:")
print(os.listdir('images'))



Files in 'images' directory:
['image1.jpg', 'image3.jpg', 'image2.jpg']


## **NOTE:** **How to Perform non maximum suppression given boxes and corresponding scores.**

```indices	=	cv.dnn.NMSBoxes(	bboxes, scores, score_threshold, nms_threshold[, eta[, top_k]]```



**Parameters**
- bboxes	a set of bounding boxes to apply NMS.
- scores	a set of corresponding confidences.
- score_threshold	a threshold used to filter boxes by score.
- nms_threshold	a threshold used in non maximum suppression.
indices	the kept indices of bboxes after NMS.
- eta	a coefficient in adaptive threshold formula: nms_thresholdi+1=eta⋅nms_thresholdi.
- top_k	if >0, keep at most top_k picked indices.
